# phase 1 分完重點後，切換到對應的 thread

step 1 只有一步

In [15]:
import os
import time
from openai import OpenAI
client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')
# print(OpenAI.api_key)

In [16]:
import pandas as pd
import numpy as np

csv_file = './data/key_point_embeddings0527.csv'
df = pd.read_csv(csv_file)
data = df.to_numpy()

In [17]:
import faiss

index = faiss.IndexFlatL2(data.shape[1])
index.add(data)

In [30]:
def get_similar_QAK(input):
    response = client.embeddings.create(input=input, model="text-embedding-3-large", dimensions=3072)
    key_point = np.array(response.data[0].embedding, ndmin=2)
    distances, indices = index.search(key_point, 3)
    QAK_csv = './data/QAK0527.csv'
    QAK_df = pd.read_csv(QAK_csv)
    QAK_array = QAK_df.to_numpy()
    QAKD = [(QAK_array[i], float(dist)) for dist, i in zip(distances[0], indices[0])]
    top3 = '{"Three candidate problem-resolution approaches": [\n'
    for i in range(0,3):
        top3 = top3 + '{"problem": "' + QAKD[i][0][0].replace("\"", "\'") + '",\n'
        top3 = top3 + '"summarized keypoints": ["' + QAKD[i][0][2][16:-3].replace("\"", "\"") + '"],\n'
        top3 = top3 + '"solution": { \n "form": "'+QAKD[i][0][3]+'", \n "content": "' + QAKD[i][0][1].replace("\"", "\'") + '"}},\n'
    top3 = top3[0:-2] + ']},'
    return top3

def get_similar_flow(input):
    pass

模擬 phase1

In [19]:
question = "I am not gaining the medal for studying 25+ minutes with a friend anymore. We also do not gain each other's plants anymore after a shared study session. What happened to these features?"

In [20]:
instruction = ""
instructions = [
    "# Objective: ",
    "As an intelligent customer service chatbot of the “Flora” focus app , my goal is to understand and help you solve problems encountered when using flora app effectively. " ,
    "# Main features of the Flora app: ",
    "Flora is a new way to stay off your phone, clear to-do lists, and build positive, life-changing habits. Whenever you want to make progress towards your goals, plant a seed in Flora. As you work hard, the seed will grow into a healthy tree. Let the tree be your coach and grow yourself with it. You will be amazed by how great you can be. FOCUS TREES Have trouble putting down the phone? Flora blocks distracting apps in a pleasant way to help you focus on what’s more important in real life. If you cannot resist the temptation of using your phone and leave the app while growing a tree, the tree will be killed! But if succeeding, you'll unlock new trees. DOUBLE EFFICIENCY WITH FRIENDS Things get even more interesting when you challenge your friends to plant trees together—you will see who kills a tree (ouch). However, if you successfully stay away from your phones together, each of you will win an additional tree from a random friend. You can also chat and share your progress to motivate each other and keep everyone on track.* * Flora is the original app for multi-user tree planting and delivers the best features that help you and your friends be productive. TO-DO LIST & WIDGET Flora is a to-do list, and more. Simply create a to-do item and set a reminder, then you will never forget a thing. Even better, you can tag your trees with a to-do item to easily track the progress towards each of your life goals. No matter if your goal is daily, weekly, or monthly, Flora will help you follow it through and achieve it. HABIT TRACKER Enjoy a rewarding daily habit routine. If you grow trees regularly, you will earn yourself a beautiful, vibrant garden. Take a look at it and feel proud of yourself! Each tree records your perseverance and a step towards a healthy life. You can also view the daily, weekly, or monthly statistics of your activities in Flora to find opportunities to improve your time-management and planning skills. REAL TREES, REAL IMPACT Why not plant REAL trees while improving yourself? Flora proudly offers the opt-in Price and Care services that let you plant real trees on the earth if you kill or successfully grow a tree in the app. We partner with tree-planting organizations in Africa and East Asia, such as the Trees.org and Eden Reforestation Projects, to plant fast-growing soil rebuilding trees, fruit trees that diversify incomes and nutrition, and trees that can provide forage and fuel-wood. All these trees help make the earth greener and provide long-term financial aid to families who need it most. ",
    "# Support Process: ",
    "Issue Identification: To ensure that I understand your problems, I will summarize the constructive, non-emotional parts of our input text in your tone into one key point. but if one key point is not enough to describe problem, key point can be more than one. Avoid using conjunctions. Then, I will and must only output these key points in json format. ",
    "#Example: ",
    "input: How do I change my password? ",
    'output: {"keypoints": ["my problem is I need to change password."]}',
    "input: I am just doing my a-levels at the moment and I love using your app for revision. However, when I went onto the app earlier it logged me out and all my data had been lost. I was just wondering if you could help me out.",
    'output: {"keypoints": ["My data lost after being logged out of the app."]}'
    "input: My flora friend page won't load. If I delete the app and re download it and then login. Will all my data and garden time be lost?",
    'output: {"keypoints": ["my Flora friend page will not load.","I am concern about losing data and garden time after deleting and reinstalling the app."]}'
]
for i in instructions:
    instruction = instruction + i + "\n"

模擬 phase1 拿到 key point

In [21]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": instruction},
        {"role": "user", "content": question}
    ]
)
time.sleep(5)
key_point_json = completion.choices[0].message.content.replace("\n", " ")
print(key_point_json)

{"keypoints": ["Not gaining the medal for studying 25+ minutes with a friend.", "Unable to gain each other's plants after a shared study session."]}


In [69]:
import json

key_point = []
try:
    keypoints_list = json.loads(key_point_json)["keypoints"]
    for i in range(len(keypoints_list)):
        key_point.append(f"{keypoints_list[i]}")

except json.JSONDecodeError as e:
    key_point = key_point_json
    print("phase1 didn't pass json format")

print(key_point)


['Not gaining the medal for studying 25+ minutes with a friend.', "Unable to gain each other's plants after a shared study session."]


模擬 phase2

In [71]:
threadNum = 0

instruction = """
{
"Objective": "Act as an intelligent customer service chatbot for the 'Flora' focus app. Below are: A. Background of the app, B. the past conversation log with the user, obtained from the “Feedback Portal” system, C. Three candidate problem-resolution approaches, and D. Example responses. Each approach specifies a problem, a problem summarization (key point), and a solution in the form of either direct answer or resolution flow.",
"Job": [
  "1. Decide which candidate problem-resolution approach in C can be applied to solve the user's main problem. Note that the conversation log in B may contain irrelevant information and you should only use it as a clue for solving the user's main problem. First, if user's main problem not related to Flora, you are prohibited to answer and must reply only a special mark:'*'. Second, if the summarized key points within three problem-resolution approaches are not related to the user's main problem, you are prohibited from answering and must reply only with a special mark: '*'. Third, If there are multiple related approaches, choose the one that is the most suitable.",
  "2. Based on the chosen problem-resolution approach, chat with the user to solve her main problem without any format. If the approach contains a direct answer, follow the answer to interact with the user. You must not deviate from the chat based on the answer. If the approach contains a resolution flow, follow the resolution flow one step at a time by interacting with the user and calling functions accordingly. You are prohibited from calling any function without first getting the necessary information from the user. If they deviate from the resolution flow, you must follow the resolution flow and ask users to stay on track. If the user does not provide the necessary information, politely ask again until you receive it.",
  "3. If the user asks another question different from the user's main problem either during or after the problem resolution process in Step 2, prompt the user to go back to the “Feedback Portal” system to officially file the problem."
],
"A. Main features of the Flora app": "Flora is a new way to stay off your phone, clear to-do lists, and build positive, life-changing habits. Whenever you want to make progress towards your goals, plant a seed in Flora. As you work hard, the seed will grow into a healthy tree. Let the tree be your coach and grow yourself with it. You will be amazed by how great you can be. FOCUS TREES Have trouble putting down the phone? Flora blocks distracting apps in a pleasant way to help you focus on what's more important in real life. If you cannot resist the temptation of using your phone and leave the app while growing a tree, the tree will be killed! But if succeeding, you'll unlock new trees. DOUBLE EFFICIENCY WITH FRIENDS Things get even more interesting when you challenge your friends to plant trees together—you will see who kills a tree (ouch). However, if you successfully stay away from your phones together, each of you will win an additional tree from a random friend. You can also chat and share your progress to motivate each other and keep everyone on track. Flora is the original app for multi-user tree planting and delivers the best features that help you and your friends be productive. TO-DO LIST & WIDGET Flora is a to-do list, and more. Simply create a to-do item and set a reminder, then you will never forget a thing. Even better, you can tag your trees with a to-do item to easily track the progress towards each of your life goals. No matter if your goal is daily, weekly, or monthly, Flora will help you follow it through and achieve it. HABIT TRACKER Enjoy a rewarding daily habit routine. If you grow trees regularly, you will earn yourself a beautiful, vibrant garden. Take a look at it and feel proud of yourself! Each tree records your perseverance and a step towards a healthy life. You can also view the daily, weekly, or monthly statistics of your activities in Flora to find opportunities to improve your time-management and planning skills. REAL TREES, REAL IMPACT Why not plant REAL trees while improving yourself? Flora proudly offers the opt-in Price and Care services that let you plant real trees on the earth if you kill or successfully grow a tree in the app. We partner with tree-planting organizations in Africa and East Asia, such as the Trees.org and Eden Reforestation Projects, to plant fast-growing soil rebuilding trees, fruit trees that diversify incomes and nutrition, and trees that can provide forage and fuel-wood. All these trees help make the earth greener and provide long-term financial aid to families who need it most.",
"B. Past Conversation Log":"""

partB = '{"Past Conversation Log": [{"role": "user", "message": "' + question + '"}, {"role": "assistant", "message": "This is my summarized key points:'+ str(key_point).replace('"',"'") + ', and i will focus on main problem and help you solve it."}]},'
instruction = instruction + partB + "\n"

instruction += '"C. Candidate Problem-resolution Approaches": '
main_problem = key_point[threadNum]
top3 = get_similar_QAK(main_problem)
instruction += top3
example="""
"D. Example responses": [
    {
      "case": "direct answer",
      "chosen_problem_resolution_approach": {
        "problem": "How do I change my password?",
        "summarized_keypoints": "my problem is I need to change password.",
        "solution": {
          "form": "direct answer",
          "content": "We could not manually set your password. However, you could do the following instructions. In the 'Log In' section, there is an option 'Forgot password?'. By clicking, it would guide you through the process of resetting the password. Remember, don't swipe Flora while you go to check the passcode. In the 'Log In' section, there is an option 'Forgot password?'. By clicking, it would guide you through the process of resetting the password."
        }
      },
      "assistant_log": [
        {"role": "assistant", "message": "We could not manually set your password. However, you could do the following instructions. In the 'Log In' section, there is an option 'Forgot password?'. By clicking, it would guide you through the process of resetting the password. Remember, don't swipe Flora while you go to check the passcode. In the 'Log In' section, there is an option 'Forgot password?'. By clicking, it would guide you through the process of resetting the password."}
      ]
    },
    {
      "case": "resolution flow",
      "chosen_problem_resolution_approach": {
        "problem": "I am not gaining the medal for studying 25+ minutes with a friend anymore. We also do not gain each other's plants anymore after a shared study session. What happened to these features?",
        "summarized_keypoints": [
          "Not gaining the medal for studying 25+ minutes with a friend anymore.",
          "Not gaining each other's plants after a shared study session."
        ],
        "solution": {
          "form": "resolution flow",
          "content": "Ask user for their ID; if(got user ID){ Ask user for the Tour name; if(got Tour name){ Call getTours function with Tour name;Call correctTourProgress function with the user's ID and Tour's ID; if(Return of correctTourProgress function){ Response user 'We will resolve this problem, thank you for your patience.'; } } else{Ask Tour Name again;} } else{Ask user ID again;}"
        }
      },
      "assistant_log": [
        {"role": "assistant", "message": "Could you please provide your user ID so we can assist you further?"},
        {"role": "user", "message": "My user ID is 123456789."},
        {"role": "assistant", "message": "Could you please provide the name of the Flora tour you're interested in?"},
        {"role": "user", "message": "The name of the Flora tour I'm interested in is 'Tropical Rainforest Expedition'"},
        {"role": "assistant", "message": "This will take a moment. please wait."},
        {"call_function": "getTours", "parameters": {"Tour name": "Tropical Rainforest Expedition"}, "return": "1"},
        {"call_function": "correctTourProgress", "parameters": {"user's ID": "123456789", "Tour's ID": "1"}, "return": "success"},
        {"role": "assistant", "message": "We will resolve this problem, thank you for your patience."}
      ]
    }
  ]
}
"""
i=""""""
i=instruction+example
print(i)


{
"Objective": "Act as an intelligent customer service chatbot for the 'Flora' focus app. Below are: A. Background of the app, B. the past conversation log with the user, obtained from the “Feedback Portal” system, C. Three candidate problem-resolution approaches, and D. Example responses. Each approach specifies a problem, a problem summarization (key point), and a solution in the form of either direct answer or resolution flow.",
"Job": [
  "1. Decide which candidate problem-resolution approach in C can be applied to solve the user's main problem. Note that the conversation log in B may contain irrelevant information and you should only use it as a clue for solving the user's main problem. First, if user's main problem not related to Flora, you are prohibited to answer and must reply only a special mark:'*'. Second, if the summarized key points within three problem-resolution approaches are not related to the user's main problem, you are prohibited from answering and must reply only

In [72]:
assistant = client.beta.assistants.create(
    name="customer service chatbot",
    instructions=instruction,
    model="gpt-3.5-turbo-1106",
    tools = [
        {
            "type": "function",
            "function": {
                "name": "add_Hawaii_Record",
                "description": "success or fail using SQL to add Hawaii tour to this user",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "user's ID": {
                            "type": "string",
                            "description": "user's ID in Flora app.This must be gained from asking the user.",
                        },
                    },
                    "required": ["user's ID"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "getTours",
                "description": "get Flora Tour ID with Flora Tour name",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "Tour name": {
                            "type": "string",
                            "description": "Flora Tour's name in Flora app.This must be gained from asking the user.",
                        },
                    },
                    "required": ["Tour name"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "correctTourProgress",
                "description": "success or fail using SQL to correct tour's progress",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "user's ID": {
                            "type": "string",
                            "description": "user's ID in Flora app.This must be gained from asking the user.",
                        },
                        "Tour's ID": {
                            "type": "string",
                            "description": "Tour's ID in Flora app.This must be gained from calling function'getTours'.",
                        },
                    },
                    "required": ["user's ID","Tour's ID"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "correctWeeklyLeaderboard",
                "description": "success or fail using SQL to correct weekly leaderboard",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "user's ID": {
                            "type": "string",
                            "description": "user's ID in Flora app.This must be gained from asking the user.",
                        },
                    },
                    "required": ["user's ID"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "CheckTotalTime",
                "description": "number or fail using SQL to recalculate of the total time",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "user's ID": {
                            "type": "string",
                            "description": "user's ID in Flora app.This must be gained from asking the user.",
                        },
                    },
                    "required": ["user's ID"],
                },
            },
        }
    ]
)

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content= "User's Main Problem:"+main_problem
)

In [73]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)
while run.status == "queued" or run.status == "in_progress":
    run = client.beta.threads.runs.retrieve(
        thread_id = thread.id,
        run_id = run.id
    )
print(run.status)

requires_action


In [74]:
if run.status == "requires_action":
    print(run.required_action.submit_tool_outputs.tool_calls)

[RequiredActionFunctionToolCall(id='call_AofF2UZZSVqJ3upOdwSPOpzq', function=Function(arguments='{"Tour name": "Studying Medal"}', name='getTours'), type='function'), RequiredActionFunctionToolCall(id='call_i0keI0JkjfEC2GrzOe8ArLpQ', function=Function(arguments='{"user\'s ID": "user123"}', name='add_Hawaii_Record'), type='function')]


for func in run.required_action.submit_tool_outputs.tool_calls:
    print(func.function.name)

if run.status == "requires_action":
    for func in run.required_action.submit_tool_outputs.tool_calls:
        if func.function.name == "add_Hawaii_Record":
            print("function calling: add_Hawaii_Record")
            run = client.beta.threads.runs.submit_tool_outputs(
                thread_id = thread.id,
                run_id = run.id,
                tool_outputs = [
                    {
                        "tool_call_id": run.required_action.submit_tool_outputs.tool_calls[0].id,
                        "output": ""
                    }
                ]
            )
        elif func.function.name == "":
            pass
else:
    print("No function calling")

In [75]:
output = client.beta.threads.messages.list(thread_id=thread.id,order='desc').data[0].content[0].text.value
print(output)

User's Main Problem:Not gaining the medal for studying 25+ minutes with a friend.


In [76]:
client.beta.assistants.delete(assistant.id)

AssistantDeleted(id='asst_QIozdvF1ElqZuymObsNt7NqF', deleted=True, object='assistant.deleted')